In [ ]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [3]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


## ExpectationValue class

In [4]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [5]:
from qiskit.providers.aer import AerSimulator

In [6]:
backend = AerSimulator()

expval = PauliExpectationValue(ansatz, observable, backend=backend)

expval.evaluate([0, 1, 1, 2, 3, 5], shots=3000)

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=array(1.874+0.j), variance=array(11.21894+0.j), confidence_interval=[((1.78037294848518+0j), (1.9676270515148193+0j))])

In [7]:
# no parameter case

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=array(1.92773438+0.j), variance=array(11.03568649+0.j), confidence_interval=[((1.769783343689555+0j), (2.085685406310445+0j))])

### Exact simulation by SaveExpectationValueVariance

In [8]:
from qiskit.evaluators import ExactExpectationValue

In [9]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
%time expval.evaluate([0, 1, 1, 2, 3, 5])

CPU times: user 7.14 ms, sys: 1.42 ms, total: 8.56 ms
Wall time: 8.01 ms


ExpectationValueResult(value=array(1.84209213), variance=array(6.43276352), confidence_interval=None)

In [10]:
%time expval.evaluate([1, 1, 2, 3, 5, 8])

CPU times: user 2.74 ms, sys: 650 µs, total: 3.39 ms
Wall time: 3.08 ms


ExpectationValueResult(value=array(0.15611897), variance=array(5.35681431), confidence_interval=None)

### Transpile options and Run options

In [11]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=1_000_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=array(1.843214+0.j), variance=array(11.13946998+0.j), confidence_interval=[((1.8381247441566668+0j), (1.8483032558433334+0j))])

In [12]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=array(1.84960938+0.j), variance=array(11.15127599+0.j), confidence_interval=[((1.7933071149424815+0j), (1.9059116350575185+0j))])

In [13]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=array(2.23333333+0.j), variance=array(10.74182222+0.j), confidence_interval=[((1.9448731836409543+0j), (2.5217934830257125+0j))])

In [29]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
# expval.set_transpile_options(initial_layout=[1, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
expval.evaluate([0, 1, 1, 2, 3, 5])

global phase: π
               ┌────┐┌──────────────┐┌────┐┌────────┐     ┌────┐»
      q_0 -> 0 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤┌─┴─┐├────┤»
      q_1 -> 1 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘└───┘└────┘»
ancilla_0 -> 2 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 3 ─────────────────────────────────────────────────»
                                                                »
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
        c22: 2/═════════════════════════════════════════════════»
                                                                »
«               ┌──────────────┐┌────┐┌────────┐     ┌────┐┌──────────────┐»
«      q_0 -> 0 ┤ Rz(θ[2] + π) ├┤ √X ├┤ Rz(3π) ├─

ExpectationValueResult(value=array(1.48999023+0.j), variance=array(11.99808472+0.j), confidence_interval=[((1.4301348230882391+0j), (1.5498456456617609+0j))])

### Mitigator

Readout mitigator classes [#6485](https://github.com/Qiskit/qiskit-terra/pull/6485) is required


In [ ]:
amat = ...

mitigator = CompleteReadoutMitigator(amat)
PauliExpectationValue(ansatz, observable, backend=backend, mitigator=mitigator)
expval.evaluate([0, 1, 1, 2, 3, 5])